In [4]:
### Import statements

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.io import wavfile
from numpy.linalg import svd
from scipy.stats.mstats import gmean
from matplotlib import rcParams
import scipy
import os
import sys
import glob
import pickle
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Lambda
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [5]:
### Creating my Neural Network

def create_network(network_input, n_vocab):

    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(LSTM(256))
    model.add(Dense(256))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy',
              optimizer='adam')
    
    print(model.summary())
    
    return model

In [92]:
def get_notes():

    notes = []
    for file in glob.glob("BillyJoel/*.mid"):
        midi = converter.parse(file)
        print("Parsing %s" % file)
        notes_to_parse = None
        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    pickle.dump(notes, open('notes.p', 'wb'))

    return notes

In [93]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 8

    pitchnames = sorted(set(item for item in notes))

    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

In [94]:
def train_network():
    """ Train a Neural Network to generate music """
    notes = get_notes()

    n_vocab = len(set(notes))
    
    print(n_vocab)
    
    network_input, network_output = prepare_sequences(notes, n_vocab)
    
    model = create_network(network_input, n_vocab)
 
    checkpoint = ModelCheckpoint(
        "weights2-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5",
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    
    callbacks_list = [checkpoint]

    # Your line of code here
    model.fit(network_input, network_output, nb_epoch=50, callbacks=callbacks_list)
    


In [95]:
train_network()

Parsing BillyJoel/Billy_Joel-All_Shook_Up.mid
Parsing BillyJoel/Billy_Joel-Pressure.mid
Parsing BillyJoel/Billy_Joel-Its_Still_Rock_And_Roll.mid
Parsing BillyJoel/Billy_Joel-Stiletto.mid
Parsing BillyJoel/Billy_Joel-Lullabye.mid
Parsing BillyJoel/Billy_Joel-Uptown_Girl.mid
Parsing BillyJoel/Billy_Joel-NY_State_Of_Mind.mid
Parsing BillyJoel/Billy_Joel-Scenes_From_An_Italian_Restaurany.mid
Parsing BillyJoel/Billy_Joel-Big_Shot.mid
Parsing BillyJoel/Billy_Joel-A_Matter_Of_Trust.mid
Parsing BillyJoel/Billy_Joel-The_Longest_Time.mid
Parsing BillyJoel/Billy_Joel-Movin_Out.mid
Parsing BillyJoel/Billy_Joel-Mexican_Connection.mid
Parsing BillyJoel/Billy_Joel-Just_The_Way_You_Are.mid
Parsing BillyJoel/Billy_Joel-River_Of_Dreams.mid
Parsing BillyJoel/Billy_Joel-Piano_Man.mid
Parsing BillyJoel/Billy_Joel-Leave_A_Tender_Moment_Alone.mid
Parsing BillyJoel/Billy_Joel-Captain_Jack.mid
Parsing BillyJoel/Billy_Joel-Shes_Got_A_Way.mid
Parsing BillyJoel/Billy_Joel-Big_Man.mid
Parsing BillyJoel/Billy_Joel-

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/50
33279/33279 [==============================] - 150s 5ms/step - loss: 5.0198
Epoch 2/50
33279/33279 [==============================] - 144s 4ms/step - loss: 4.5872
Epoch 3/50
33279/33279 [==============================] - 140s 4ms/step - loss: 4.3525
Epoch 4/50
33279/33279 [==============================] - 137s 4ms/step - loss: 4.1292
Epoch 5/50
33279/33279 [==============================] - 136s 4ms/step - loss: 3.9092
Epoch 6/50
33279/33279 [==============================] - 136s 4ms/step - loss: 3.7179
Epoch 7/50
33279/33279 [==============================] - 136s 4ms/step - loss: 3.5436
Epoch 8/50
33279/33279 [==============================] - 178s 5ms/step - loss: 3.3789
Epoch 9/50
33279/33279 [==============================] - 159s 5ms/step - loss: 3.2177
Epoch 10/50
33279/33279 [==============================] - 136s 4ms/step - loss: 3.0716
Epoch 11/50
33279/33279 [==============================] - 136s 4ms/step - loss: 2.9316
Epoch 12/50
33279/33279 [================

In [6]:
def prepare_sequences_prediction(notes, pitchnames, n_vocab):

    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    sequence_length = 8
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # Starts the melody by picking a random sequence from the input as a starting point
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    for note_index in range(200):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        
        ### Copy the line below from your above implementation.
        prediction = model.predict(prediction_input)
        probs = []
        total = 0
        
        for j in range(0, len(prediction[0])):
            total += prediction[0, j]
            
        for j in range(0, len(prediction[0])):
            probs.append((prediction[0, j] / total))

        index = np.random.choice(n_vocab, 1, replace=False, p=probs)

        result = int_to_note[index[0]]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [7]:
def generate():
    notes = pickle.load(open('notes.p', 'rb'))
    pitchnames = sorted(set(item for item in notes))
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences_prediction(notes, pitchnames, n_vocab)
    model = create_network(normalized_input, n_vocab)
    
    ### Add a line to load the weights here
    model.load_weights("weights2-improvement-50-0.7955-bigger.hdf5")
    
    
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

In [ ]:
def create_midi(prediction_output):
    offset = 0
    output_notes = []
    for pattern in prediction_output:
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        offset += 0.5
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='test_output.mid')
    
generate()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 8, 512)            1052672   
_________________________________________________________________
dropout_1 (Dropout)          (None, 8, 512)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 8, 512)            2099200   
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 512)            2048      
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 512)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               787456    